In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
from copy import copy
pd.options.plotting.backend = 'plotly'
from plotly import graph_objects as go
from aust_covid.inputs import load_raw_pop_data
from general_utils.tex_utils import StandardTexDoc
PROJECT_PATH = Path().resolve().parent

In [ ]:
def get_ifrs(tex_doc):
    description = ''
    tex_doc.add_line(description, 'Parameters', subsection='Infection fatality rates')
    
    # Raw data from O'Driscoll, 5-year age bands
    odriscoll = pd.Series(
        {
            0: 0.003,
            5: 0.001,
            10: 0.001,
            15: 0.003,
            20: 0.006,
            25: 0.013,
            30: 0.024,
            35: 0.04,
            40: 0.075,
            45: 0.121,
            50: 0.207,
            55: 0.323,
            60: 0.456,
            65: 1.075,
            70: 1.674,
            75: 3.203,
            80: 8.292,
        }
    ) / 100.0
    odriscoll.index = odriscoll.index + 2.5

    # Raw data from Erikstrup
    erikstrup = pd.Series(
        {
            (17 + 36) / 2: 2.6,
            (36 + 51) / 2: 5.8,
            (51 + 61) / 2: 14.6,
            (61 + 73) / 2: 24.6,
        }
    ) / 1e5

    # Most comparable upper and lower age group ratio
    lower_ratio = erikstrup[26.5] / odriscoll[27.5]
    upper_ratio = erikstrup[67.0] / odriscoll[67.5]

    # Apply the ratios to the upper and missing age groups without estimates from Erikstrup
    lower_adjusted = odriscoll[ :17.5] * lower_ratio
    upper_adjusted = odriscoll[72.5: ] * upper_ratio

    # Combine extrapolated estimates with Erikstrup
    combined = pd.concat([lower_adjusted, erikstrup, upper_adjusted])

    # Modelled breakpoints (lower values rather than midpoints of age groups)
    age_mid_points = np.linspace(2.5, 72.5, 15)
    final_values = pd.Series(np.interp(age_mid_points, combined.index, combined), index=age_mid_points)

    # Proportion of the 75+ age group to take from the 80+ estimate
    pops = load_raw_pop_data('31010do002_202206.xlsx').sum(axis=1)
    prop_75_over_80 = pops['80-84': ].sum() / pops['75-79': ].sum()
    final_values[77.5] = combined[82.5] * prop_75_over_80 + combined[77.5] * (1.0 - prop_75_over_80)

    model_breakpoint_values = copy(final_values)
    model_breakpoint_values.index = final_values.index - 2.5
    
    return odriscoll, erikstrup, upper_adjusted, lower_adjusted, combined, final_values, model_breakpoint_values

In [ ]:
app_doc = StandardTexDoc(PROJECT_PATH / 'supplement', 'supplement', "Australia's 2023 Omicron Waves Supplement", 'austcovid')
odriscoll, erikstrup, upper_adjusted, lower_adjusted, combined, final_values, model_breakpoint_values = get_ifrs(app_doc)
scaling_factor = 0.04
fig = go.Figure()
fig.add_trace(go.Scatter(x=odriscoll.index, y=odriscoll, name="O'Driscoll"))
fig.add_trace(go.Scatter(x=erikstrup.index, y=erikstrup, name='Erikstrup'))
fig.add_trace(go.Scatter(x=upper_adjusted.index, y=upper_adjusted, name="Upper adjusted O'Driscoll"))
fig.add_trace(go.Scatter(x=lower_adjusted.index, y=lower_adjusted, name="Lower adjusted O'Driscoll"))
fig.add_trace(go.Scatter(x=combined.index, y=combined, name="Combined Erikstrup/O'Driscoll"))
fig.add_trace(go.Scatter(x=final_values.index, y=final_values, name='Combined and interpolated'))
fig.add_trace(go.Scatter(x=model_breakpoint_values.index, y=model_breakpoint_values, name='Values by model breakpoints'))
fig.update_yaxes(type='log')